In [22]:
import pandas as pd

df = pd.read_parquet('1.limpo.parquet').astype({'data': 'datetime64[ns]'})
len(df)

10298930

In [ ]:
def _remover_colunas(df):
    novo_df = df.copy()
    return novo_df.drop(columns=["Recuperadosnovos",
                                 "emAcompanhamentoNovos"])


def _obter_estacao(data):
    ano = data.year
    datas_estacoes = {
        "verao": pd.Timestamp(f"{ano}-12-21"),
        "outono": pd.Timestamp(f"{ano}-3-21"),
        "inverno": pd.Timestamp(f"{ano}-6-21"),
        "primavera": pd.Timestamp(f"{ano}-9-21"),
    }

    if data >= datas_estacoes["verao"] or data < datas_estacoes["outono"]:
        return "Verão"
    elif data >= datas_estacoes["outono"] and data < datas_estacoes["inverno"]:
        return "Outono"
    elif data >= datas_estacoes["inverno"] and data < datas_estacoes["primavera"]:
        return "Inverno"
    else:
        return "Primavera"


def _traduzir_dias_da_semana(ingles):
    portugues = {
        "Sunday": "Domingo",
        "Monday": "Segunda-feira",
        "Tuesday": "Terça-feira",
        "Wednesday": "Quarta-feira",
        "Thursday": "Quinta-feira",
        "Friday": "Sexta-feira",
        "Saturday": "Sábado"
    }

    return portugues[ingles]


def _traduzir_mes(ingles):
    portuguese = {
        "January": "Janeiro",
        "February": "Fevereiro",
        "March": "Março",
        "April": "Abril",
        "May": "Maio",
        "June": "Junho",
        "July": "Julho",
        "August": "Agosto",
        "September": "Setembro",
        "October": "Outubro",
        "November": "Novembro",
        "December": "Dezembro"
    }

    return portuguese[ingles]


def _adicionar_feature_datas(df):
    novo_df = df.copy()

    novo_df["ano"] = novo_df.data.dt.year
    novo_df["mes"] = novo_df.data.dt.month_name()
    novo_df["mes_numerico"] = novo_df.data.dt.month
    novo_df["mes_traduzido"] = novo_df["mes"].apply(_traduzir_mes)
    novo_df["dia_semana"] = novo_df.data.dt.day_name()
    novo_df["dia_semana_traduzido"] = novo_df["dia_semana"].apply(
        _traduzir_dias_da_semana)
    novo_df["dia_semana_numerico"] = novo_df.data.dt.day_of_week
    novo_df["estacao"] = novo_df["data"].apply(_obter_estacao)

    return novo_df


def _limpar_nomes_de_colunas(df):
    colunas = {
        'regiao': 'regiao',
        'estado': 'estado',
        'municipio': 'municipio',
        'coduf': 'coduf',
        'codmun': 'codmun',
        'codRegiaoSaude': 'cod_regiao_saude',
        'nomeRegiaoSaude': 'nome_regiao_saude',
        'data': 'data',
        'semanaEpi': 'semana_epi',
        'populacaoTCU2019': 'populacao_tcu_2019',
        'casosAcumulado': 'casos_acumulados',
        'casosNovos': 'casos_novos',
        'novos_casos_novos': 'novos_casos_novos',
        'obitosAcumulado': 'obitos_acumulados',
        'obitosNovos': 'obitos_novos',
        'Recuperadosnovos': 'recuperados_novos',
        'emAcompanhamentoNovos': 'em_acompanhamento_novos',
        'interior/metropolitana': 'interior_metropolitana',
        'novos_casos_acumulados': 'novos_casos_acumulados'
    }
    return df.rename(columns=colunas)

def _adicionar_lag_features(df):
    # assumindo que df é de apenas um município único
    # df está ordenado por data

    df["novos_casos_novos_lag_1"] = df['novos_casos_novos'].shift(1)
    df["obitos_novos_lag_1"] = df['obitos_novos'].shift(1)

    df["novos_casos_novos_lag_7"] = df['novos_casos_novos'].shift(7)
    df["obitos_novos_lag_7"] = df['obitos_novos'].shift(7)

    df["novos_casos_novos_lag_14"] = df['novos_casos_novos'].shift(14)
    df["obitos_novos_lag_14"] = df['obitos_novos'].shift(14)

    return df

def _adicionar_media_movel_7(df):
    # assumindo que df é de apenas um município único
    # df está ordenado por data

    df['novos_casos_novos_media_movel_7'] = df['novos_casos_novos'].transform(lambda x: x.shift(1).rolling(7).mean())
    df['obitosNovos_media_movel_7'] = df['obitos_novos'].transform(lambda x: x.shift(1).rolling(7).mean())

    return df


In [42]:
min_df = _limpar_nomes_de_colunas(df.head(300))
_adicionar_media_movel_7(min_df)[295:300]

# df.head(5)

,regiao,estado,municipio,coduf,codmun,cod_regiao_saude,nome_regiao_saude,data,semana_epi,populacao_tcu_2019,casos_acumulados,casos_novos,obitos_acumulados,obitos_novos,recuperados_novos,em_acompanhamento_novos,interior_metropolitana,novos_casos_novos,novos_casos_acumulados,novos_casos_novos_media_movel_7
295,Centro-Oeste,GO,Abadia de Goiás,52,520005.0,52001.0,CENTRAL,2021-01-16,2,8773.0,786.0,7,9,0,NaN,NaN,1.0,7,786,1.285714
296,Centro-Oeste,GO,Abadia de Goiás,52,520005.0,52001.0,CENTRAL,2021-01-17,3,8773.0,785.0,-1,10,1,NaN,NaN,1.0,-1,785,1.857143
297,Centro-Oeste,GO,Abadia de Goiás,52,520005.0,52001.0,CENTRAL,2021-01-18,3,8773.0,786.0,1,10,0,NaN,NaN,1.0,1,786,1.714286
298,Centro-Oeste,GO,Abadia de Goiás,52,520005.0,52001.0,CENTRAL,2021-01-19,3,8773.0,789.0,3,10,0,NaN,NaN,1.0,3,789,1.857143
299,Centro-Oeste,GO,Abadia de Goiás,52,520005.0,52001.0,CENTRAL,2021-01-20,3,8773.0,789.0,0,10,0,NaN,NaN,1.0,0,789,2.285714


In [ ]:
def _rodar_engenharia_de_atributos(df):
    df = _remover_colunas(df)
    df = _limpar_nomes_de_colunas(df)
    df = _adicionar_feature_datas(df)
    df = _adicionar_lag_features(df)

    return df

df = _rodar_engenharia_de_atributos(df)
len(df)

10298930

In [4]:
df.to_parquet('2.atributos.parquet', index=False)

In [5]:
df = pd.read_parquet('2.atributos.parquet')

df.head()

,regiao,estado,municipio,coduf,codmun,cod_regiao_saude,nome_regiao_saude,data,semana_epi,populacao_tcu_2019,...,casos_novos,obitos_acumulados,obitos_novos,interior_metropolitana,novos_casos_acumulados,mes,mes_traduzido,dia_semana,dia_semana_traduzido,estacao
0,Centro-Oeste,GO,Abadia de Goiás,52,520005.0,52001.0,CENTRAL,2020-03-27,13,8773.0,...,0,0,0,1.0,0,March,Março,Friday,Sexta-feira,Outono
1,Centro-Oeste,GO,Abadia de Goiás,52,520005.0,52001.0,CENTRAL,2020-03-28,13,8773.0,...,0,0,0,1.0,0,March,Março,Saturday,Sábado,Outono
2,Centro-Oeste,GO,Abadia de Goiás,52,520005.0,52001.0,CENTRAL,2020-03-29,14,8773.0,...,0,0,0,1.0,0,March,Março,Sunday,Domingo,Outono
3,Centro-Oeste,GO,Abadia de Goiás,52,520005.0,52001.0,CENTRAL,2020-03-30,14,8773.0,...,0,0,0,1.0,0,March,Março,Monday,Segunda-feira,Outono
4,Centro-Oeste,GO,Abadia de Goiás,52,520005.0,52001.0,CENTRAL,2020-03-31,14,8773.0,...,0,0,0,1.0,0,March,Março,Tuesday,Terça-feira,Outono
